# Following along with FastAI Intro to Machine Learning Lesson 1

In [1]:
#from fastai.imports import *
#from fastai.structured import *
import os
import math
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn.ensemble import forest
import re
from sklearn import metrics
import numpy as np

C:\Users\Aaron\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [6]:
PATH = "data/bulldozers/"

In [7]:
!ls {PATH}

Train.csv
Train.zip


In [8]:
!head data/bulldozers/Train.csv

SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
1139246,66000,999089,3157,121,3,2004,68,Low,11/16/2006 0:00,521D,521,D,,,,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,,EROPS w AC,None or Unspecified,,None or Unspecified,,,,,,,,2 Valve,,,,,None or Unspecified,None or Unspecified,,,,,,,,,,,,,Standard,Conventional
1139248,57000,117657,77,121,3,1996,4640,Lo

In [9]:
df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory = False,
                    parse_dates = ['saledate'])

# Side Note:

In [10]:
df_raw

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1139255,26500,1001274,4605,121,3.0,2004,508.0,Low,2008-12-18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1139256,21000,772701,1937,121,3.0,1993,11540.0,High,2004-08-26,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
7,1139261,27000,902002,3539,121,3.0,2001,4883.0,High,2005-11-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1139272,21500,1036251,36003,121,3.0,2008,302.0,Low,2009-08-27,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
9,1139275,65000,1016474,3883,121,3.0,1000,20700.0,Medium,2007-08-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


In [11]:
df_raw['saledate'].head(3)

0   2006-11-16
1   2004-03-26
2   2004-02-26
Name: saledate, dtype: datetime64[ns]

#### The above uses a pandas function that will read the csv, and will translate the saledate column in a date format

In [12]:
df_raw2 = pd.read_csv(f'{PATH}Train.csv', low_memory = False)

In [13]:
df_raw2

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1139255,26500,1001274,4605,121,3.0,2004,508.0,Low,12/18/2008 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1139256,21000,772701,1937,121,3.0,1993,11540.0,High,8/26/2004 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
7,1139261,27000,902002,3539,121,3.0,2001,4883.0,High,11/17/2005 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1139272,21500,1036251,36003,121,3.0,2008,302.0,Low,8/27/2009 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
9,1139275,65000,1016474,3883,121,3.0,1000,20700.0,Medium,8/9/2007 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


In [14]:
df_raw2['saledate'].head(3)

0    11/16/2006 0:00
1     3/26/2004 0:00
2     2/26/2004 0:00
Name: saledate, dtype: object

#### You can see that the saledate has no clear object type unless you specify the parse_date feature
# Back to the lesson

In [15]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000):# will display 1000 rows max instead of otherwise
        with pd.option_context("display.max_columns", 1000):#will display 100 columns instead of otherwise
            display(df)

In [16]:
display_all(df_raw.tail().transpose())#shows last few rows and switches columns with rows

,401120,401121,401122,401123,401124
SalesID,6333336,6333337,6333338,6333341,6333342
SalePrice,10500,11000,11500,9000,7750
MachineID,1840702,1830472,1887659,1903570,1926965
ModelID,21439,21439,21439,21435,21435
datasource,149,149,149,149,149
auctioneerID,1,1,1,2,2
YearMade,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
saledate,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00


### EDA = Exploratory Data Analysis. Theoretically, never. Practically, tiny bit.

### Evaluation of the metric is on RMSLE(root mean squared log error) which instead focuses on the ratio difference and not absolute difference

#### since this is a RMSLE, then we need the log of the prices, not the actual prices

In [17]:
df_raw.SalePrice.head(5)

0    66000
1    57000
2    10000
3    38500
4    11000
Name: SalePrice, dtype: int64

In [18]:
df_raw.SalePrice = np.log(df_raw.SalePrice) #O(N*log(N)) time and uses ln not log base 10

In [19]:
df_raw.SalePrice.head(5)

0    11.097410
1    10.950807
2     9.210340
3    10.558414
4     9.305651
Name: SalePrice, dtype: float64

### As you can see, we are now looking at the ln of the prices

# Creation of a Random Forest

## This is a universal technique for prediction, and is a standard. It doesn't overfit usually, but does have that issue and can be intercepted easily. No statistical assumptions

In [20]:
m = RandomForestRegressor(n_jobs = -1)#basically means unlimited amount of processors can parallelize for it. From sklearn
m.fit(df_raw.drop('SalePrice', axis=1), df_raw.SalePrice)
#drops the salePrice column(1 means column, 0 means row) and has it become the y-axis

ValueError: could not convert string to float: 'Low'

### This did not work because of the mixture of strings with numbers. The dataset has a mix of *Continuous* and *Categorical* vars. The biggest issue is the date-time, df_raw.saledate, 

### Regressers predict some kind of number continuously, rather than categorical prediction

# Feature Engineering

### Inside a date we have month, day, year, holiday, weekend, weather(if we are willing to map with historical weather data), season, weekend, weekday, local events, sporting events, all sorts!

### This of course cannot be done by ML algorithms, and this is where I step in and work with it.

In [21]:
def add_datepart(df, fldname):
    fld= df[fldname] 
    #not df.fldname, that would just grab a field that states 'fldname'. also safer and should do more than this.
    targ_pre = re.sub('[Dd]ate$', '', fldname)#regex removes 'date' from fldname
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek','Dayofyear',#goes through each string
              'Is_month_end','Is_month_start', 'Is_quarter_end', 'Is_year_end', 'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
        #tries to find an attribute with the same name inside each object(like Year attr)
    df[targ_pre+'Elapsed'] = (fld - fld.min()).dt.days
    df.drop(fldname, axis=1, inplace=True)
        

In [22]:
fld = df_raw.saledate

In [23]:
fld.head(4)

0   2006-11-16
1   2004-03-26
2   2004-02-26
3   2011-05-19
Name: saledate, dtype: datetime64[ns]

In [24]:
fld.dt.year.head(5)

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saledate, dtype: int64

### The above shows that datetime64 hides these fields unless explicity called using'.dt.'. Use code completion (Tab at the period) to see the options that you have, and the function 'add_datepart' will pull any relevent ones!

In [25]:
add_datepart(df_raw, 'saledate')
df_raw.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

In [26]:
df_raw.head().transpose()

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,11.0974,10.9508,9.21034,10.5584,9.30565
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,Low,Low,High,High,Medium
fiModelDesc,521D,950FII,226,PC120-6E,S175


### As you can see, it will create new columns for consideration. There is a standard library for doing this right away, but it may nor be ideal

### But now we have other strings we need to remove, such as standard, conventional, low, medium, high, and 'NaN'. We can do one hot encoding/mapping to create integers out of the categorical values

In [27]:
def train_cats(df):
    for n, c in df.items():#row, object
        if is_string_dtype(c):#checks if string.
            df[n] = c.astype('category').cat.as_ordered()#sets the categorical value for ordering.

In [28]:
train_cats(df_raw)

In [29]:
df_raw.UsageBand.cat.categories#ordered as is instead of setting

Index(['High', 'Low', 'Medium'], dtype='object')

In [30]:
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered = True, inplace = True)

In [31]:
df_raw.UsageBand.cat.codes.head(20) #shows the numerical value instead of the categorical one

0     2
1     2
2     0
3     0
4     1
5     2
6     0
7     0
8     2
9     1
10    1
11    2
12   -1
13    1
14    2
15    1
16    2
17   -1
18    2
19    1
dtype: int8

#### Without doing anything, a blank slot will be called NaN categorically, but will have a -1 numerically.

### We have more! We still have missing values that won't be passed to the randomForest

In [32]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

#### feather format for saving a file quickly, extremely efficient and extremely new. Basically saves the exact memory specification as the binary data on disk

In [33]:
os.makedirs('tmp', exist_ok =True)
df_raw.to_feather('tmp/raw')

### Now we replace the strings with their numeric codes, handle missing values, and split the dependent variable(saleDate) int a separate variable

In [38]:
def proc_df(df, y_fld, skip_flds =None, do_scale = False,
           preproc_fn = None, max_n_cat = None, subset = None):
    if not skip_flds:#copies df, grabs y values, drops the original, and then does fix_missing
        skip_flds = []
    if subset:
        df= get_sample(df, subset)
    df = df.copy()
    if preproc_fn:
        preproc_fn(df)
    y = df[y_fld].values
    df.drop(skip_flds+[y_fld], axis=1, inplace =True)
    
    for n, c in df.items():
        fix_missing(df, c, n)
    if do_scale:
        mapper = scale_vars(df)
    for n, c in df.items():
        numericalize(df, c, n, max_n_cat)
    res = [pd.get_dummies(df, dummy_na = True), y]#discuss later
    if not do_scale: return res
    return res + [mapper]

In [39]:
def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum():
            df[name+'_na'] = pd.isnull(col)
        df[name] = col.fillna(col.median())# replaced with median, and replaces with new column telling what's missing

In [40]:
def numericalize(df, col, name, max_n_cat):
    if not is_numeric_dtype(col) and (max_n_cat is None or col.nunique()>max_n_cat):
        df[name] = col.cat.codes+1# if it isn't numeric, we replace with its codes.
    

In [43]:
df, y =proc_df(df_raw, 'SalePrice')

In [44]:
df.columns

Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'fiModelDesc',
       'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries', 'fiModelDescriptor',
       'ProductSize', 'fiProductClassDesc', 'state', 'ProductGroup',
       'ProductGroupDesc', 'Drive_System', 'Enclosure', 'Forks', 'Pad_Type',
       'Ride_Control', 'Stick', 'Transmission', 'Turbocharged',
       'Blade_Extension', 'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower',
       'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control',
       'Tire_Size', 'Coupler', 'Coupler_System', 'Grouser_Tracks',
       'Hydraulics_Flow', 'Track_Type', 'Undercarriage_Pad_Width',
       'Stick_Length', 'Thumb', 'Pattern_Changer', 'Grouser_Type',
       'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',
       'saleI

### Sale Price is no longer part of df, it is now in the new variable 'y'. we also have extra columns of [column].na

In [45]:
df.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,...,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_year_end,saleIs_year_start,saleElapsed,auctioneerID_na,MachineHoursCurrentMeter_na
0,1139246,999089,3157,121,3.0,2004,68.0,3,950,296,...,3,320,False,False,False,False,False,6512,False,False
1,1139248,117657,77,121,3.0,1996,4640.0,3,1725,527,...,4,86,False,False,False,False,False,5547,False,False
2,1139249,434808,7009,121,3.0,2001,2838.0,1,331,110,...,3,57,False,False,False,False,False,5518,False,False
3,1139251,1026470,332,121,3.0,2001,3486.0,1,3674,1375,...,3,139,False,False,False,False,False,8157,False,False
4,1139253,1057373,17311,121,3.0,2007,722.0,2,4208,1529,...,3,204,False,False,False,False,False,7492,False,False


###  Anything boolean is treated as a number 

# Time to build the random Forest!(trivially parallelizable)

In [46]:
m = RandomForestRegressor(n_jobs = -1)

In [47]:
m.fit(df, y)
m.score(df, y)

0.9831159894556205

### scores it on r^2, which is between 0 and 1. Looks great, but we don't know if there is overfitting or not.

## This is solved by splitting data into a validation set and a training set.

In [48]:
def split_vals(a, n):
    return a[:n].copy(), a[n: ].copy()

n_valid = 12000 #same as Kaggle's test size
n_trn = len(df)- n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
Y_train, Y_valid = split_vals(y, n_trn)

X_train.shape, Y_train.shape, X_valid.shape

((389125, 65), (389125,), (12000, 65))

In [49]:
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())
def print_score(m):
    res = [rmse(m.predict(X_train), Y_train), rmse(m.predict(X_valid), Y_valid),
          m.score(X_train, Y_train), m.score(X_valid, Y_valid)]
    if hasattr(m, 'oob_score_'):
        res.append(m.oob_score_)
    print(res)

In [52]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, Y_train)
print_score(m)

Wall time: 14 s
[0.09032853679213929, 0.24735209120950322, 0.9829476288501068, 0.8907354838804902]


### The 2nd score given is the rmse prediction with the valid dation set,  the lower the number the better and , giving us an idea of what to expect on testing set. The last number is using r^2, with closer to 1 being better.